### Save Cleaned Data to SQLite

In this optional enhancement module, you'll learn how to persist your cleaned and filtered data using Python's built-in `sqlite3` module.

This enables you to:
- Save your cleaned dataset to a local SQLite database
- Reuse the data in your dashboards or other apps
- Avoid repeated processing or scraping


In [23]:
# Step 1: Import libraries
import pandas as pd
import sqlite3

In [24]:
print(sqlite3.sqlite_version)

3.45.1


In [25]:
# Step 2: Load the dataset: 'largest-companies.csv'
comp_df = pd.read_csv('../csv/largest-companies.csv')
print(comp_df.shape)
comp_df.head()

(5943, 6)


,Rank,Name,Symbol,marketcap,price (USD),country
0,1,Apple,AAPL,2801388224512,171.66,United States
1,2,Microsoft,MSFT,2256182968320,300.95,United States
2,3,Saudi Aramco,2222.SR,1973864961187,9.88,Saudi Arabia
3,4,Alphabet (Google),GOOG,1838538817536,2778.76,United States
4,5,Amazon,AMZN,1607290585088,3158.71,United States


In [26]:
# drop Rank col
comp_df.drop(columns='Rank',inplace=True)
print(comp_df.shape)
comp_df.head()

(5943, 5)


,Name,Symbol,marketcap,price (USD),country
0,Apple,AAPL,2801388224512,171.66,United States
1,Microsoft,MSFT,2256182968320,300.95,United States
2,Saudi Aramco,2222.SR,1973864961187,9.88,Saudi Arabia
3,Alphabet (Google),GOOG,1838538817536,2778.76,United States
4,Amazon,AMZN,1607290585088,3158.71,United States


In [27]:
# rename "Name" and "Symbol" to lowercase
comp_df.rename(columns={
    "Name":"name",
    "Symbol":"symbol"
},inplace=True)
print(comp_df.shape)
comp_df.head()

(5943, 5)


,name,symbol,marketcap,price (USD),country
0,Apple,AAPL,2801388224512,171.66,United States
1,Microsoft,MSFT,2256182968320,300.95,United States
2,Saudi Aramco,2222.SR,1973864961187,9.88,Saudi Arabia
3,Alphabet (Google),GOOG,1838538817536,2778.76,United States
4,Amazon,AMZN,1607290585088,3158.71,United States


In [28]:
# Step 3: Connect to a local SQLite database file
# if it does not exist, the db file will be made
# the connect() method returns a connection object
# which has methods and properties for doing CRUD ops
# Create, Read, Update, Delete
conn = sqlite3.connect("../companies.db")
# L@@K: the companies.db file is in your folder BUT it is empty

In [29]:
# Creates the .db file if it doesn't exist 
# that is: save the df to sqlite3 as a db
# next save the df to the db BUT can't just lie loose
# the df data must go inside a TABLE
# df.to_sql(table_name, conn)
# save the df to a "company" table in the "companies.db" db
comp_df.to_sql("company", conn, if_exists="replace", index=False)
# if_exists="replace" replaces any existing "company" table with this one
# index=False means do NOT make a column in the table to store index (0-5942)
# L@@K: it returned the number of rows written to the companies table!

5943

### check out db in terminal
- In terminal, type: **sqlite3 companies.db**
- type **.tables** to see list of tables
- type **.schema company** to seee schema (structure) of table
- query first 10 rows: **SELECT * FROM company LIMIT 10;**
- type this to leave DB mode: **.quit**

Now your data is safely stored in `companies.db` and can be queried anytime, including by Dash apps.

In [30]:
# Step 4: Read the data back from the database
# by convention, SQL keywords are UPPERCASE
# df = pd.read_sql("SQL_COMMAND", conn)
comp_db_df = pd.read_sql("SELECT * FROM company", conn)
# "select * from companies" would also work

In [31]:
# Step 5: Always close the connection
conn.close()

In [32]:
# verify that the SQL data loaded into the new DF:
print(comp_db_df.shape)
comp_db_df.head()

(5943, 5)


,name,symbol,marketcap,price (USD),country
0,Apple,AAPL,2801388224512,171.66,United States
1,Microsoft,MSFT,2256182968320,300.95,United States
2,Saudi Aramco,2222.SR,1973864961187,9.88,Saudi Arabia
3,Alphabet (Google),GOOG,1838538817536,2778.76,United States
4,Amazon,AMZN,1607290585088,3158.71,United States


In [33]:
# LAB: Following the above sequence, make an sqlite3 db from the toys csv
# call the db "toys-db" and call the table inside it "toys"
# in other words, do not put a new table inside companies db -- this is a new db BUT steps are the SAME as above here
# steps: 
# a.) load the toys CSV into a DF
# b.) make a new SQL DB (empty)
# c.) save toys DF to SQL DB table
# d.) read toys SQL database back up into new DF
#.    the new DF has the same data as orig DF
# HINT: IT is EXACTLY the same procedure as w companies
#.      DO NOT make detour into terminal--stay in Python
#.      This can all be done in a few one or two line code cells

# a.) load the toys CSV into a DF
toys_df = pd.read_csv('../csv/toy-sales.csv')
print(toys_df.shape)
toys_df.head()

(636, 5)


,date,region,product,revenue,units
0,2023-01-01,North,Widget,112.66,22
1,2023-01-01,North,Gadget,92.29,23
2,2023-01-01,North,Doodad,63.00,11
3,2023-01-01,South,Widget,105.07,23
4,2023-01-01,South,Gadget,71.16,15


In [34]:
# b.) make a new SQL DB (empty)
conn_toys = sqlite3.connect("../toys.db")

In [35]:
# c.) save toys DF to SQL DB table
toys_df.to_sql("toys", conn_toys, if_exists="replace", index=False)

636

In [36]:
# d.) read toys SQL database back up into new DF
toys_db_df = pd.read_sql("SELECT * FROM toys", conn_toys)
print(toys_db_df.shape)
toys_db_df.head()

(636, 5)


,date,region,product,revenue,units
0,2023-01-01,North,Widget,112.66,22
1,2023-01-01,North,Gadget,92.29,23
2,2023-01-01,North,Doodad,63.00,11
3,2023-01-01,South,Widget,105.07,23
4,2023-01-01,South,Gadget,71.16,15


In [37]:
# e.) close the connection
conn_toys.close()